In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from linearmodels import PanelOLS
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import ResidualsPlot
from scipy import stats

In [2]:
joint = pd.read_excel('joint_snap.xlsx')

In [5]:
#convert the month column into correct form
joint['Month'] = pd.to_datetime(joint['Month'])
age = (joint['Month'] - joint['Inception Date'])/np.timedelta64(1,'M')
age = [int(i) for i in age]
joint['Age'] = age
#joint.dropna(subset=["esg_score"], inplace=True)

#convert the esb_l from string to float
esb_l = pd.DataFrame({'Equity Style Box (Long)':['Large Value', 'Large Growth', 'Large Blend', 'Mid Value', 'Mid Growth', 'Mid Blend','Small Value','Small Growth','Small Blend'], 'esb_l': [1, 2, 3, 4, 5, 6, 7, 8, 9]})
joint_new = joint.join(esb_l.set_index('Equity Style Box (Long)'), on='Equity Style Box (Long)')

#relative_flow is the size divided by flow value
flow = joint_new["Fund Flow"]
size = joint_new['Fund Size']
relative_flow = flow/size
joint_new['relative_flow'] = relative_flow

joint_new = joint_new.rename(columns={"Fund Sustainability": "esg_score"})

In [6]:
#build a new column, in which the relative_flow is the value of last month
flow_org = joint_new['relative_flow']
flow_nxt = [np.nan]*len(flow_org)

verschiebung = 1

for i in range(3276):
    for j in range(verschiebung,92):
         flow_nxt[92*i+j-verschiebung] =flow_org[92*i+j]
            
joint_new['relative_flow_nxt'] = flow_nxt

In [7]:
joint_new

,FundId,Month,Fund Size,Fund Flow,esg_score,capm_Performance,performance_Three_Factor,performance_Four_Factor,Inception Date,Equity Style Box (Long),Age,size_log,esb_l,relative_flow,relative_flow_nxt
0,FS00008SZD,2012-01-01,2.609599e+08,4.379443e+07,NaN,1.507391,3.652442,1.841484,2009-01-28,Large Growth,35,8.416574,2.0,0.167821,0.086092
1,FS00008SZD,2012-02-01,2.988403e+08,2.572761e+07,NaN,1.544497,1.220171,1.155288,2009-01-28,Large Growth,36,8.475439,2.0,0.086092,0.005074
2,FS00008SZD,2012-03-01,3.071028e+08,1.558167e+06,NaN,4.025839,0.078026,0.963080,2009-01-28,Large Growth,37,8.487284,2.0,0.005074,0.109079
3,FS00008SZD,2012-04-01,3.440745e+08,3.753147e+07,NaN,2.647697,0.989935,1.171659,2009-01-28,Large Growth,38,8.536653,2.0,0.109079,0.129341
4,FS00008SZD,2012-05-01,3.771187e+08,4.877692e+07,NaN,-0.066138,-0.807304,-0.354775,2009-01-28,Large Growth,39,8.576478,2.0,0.129341,0.027948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301387,FSUSA09WNF,2019-04-01,8.137770e+06,-1.487095e+03,NaN,-3.067829,-3.546688,-4.150616,2008-11-11,Large Blend,124,6.910505,3.0,-0.000183,0.000101
301388,FSUSA09WNF,2019-05-01,7.979041e+06,8.084123e+02,NaN,3.666930,4.017213,5.035920,2008-11-11,Large Blend,125,6.901951,3.0,0.000101,0.000238
301389,FSUSA09WNF,2019-06-01,8.651099e+06,2.060187e+03,40.91,4.207674,3.772864,3.199166,2008-11-11,Large Blend,126,6.937071,3.0,0.000238,-0.000259
301390,FSUSA09WNF,2019-07-01,8.737163e+06,-2.267210e+03,NaN,1.134835,4.536452,4.007588,2008-11-11,Large Blend,127,6.941370,3.0,-0.000259,0.000258


In [25]:
#build a new column, performance * esg_score
#performance_score_1 = [joint_new['capm_Performance'][i] * joint_new['esg_score'] for i in range(len(joint_new))]
performance_score_4 = joint_new['capm_Performance']*joint_new['esg_score'] / 100

In [26]:
joint_new['esg_performance'] = performance_score_4

In [27]:
#joint_new[joint_new['Month']-pd.to_datetime(2015-10-12)>0]
diff = (joint_new['Month'] - pd.Timestamp(2015,10,12))/np.timedelta64(1,'M')
joint_new['date_diff'] = diff
front_month = joint_new[joint_new['date_diff']<0]
behind_month = joint_new[joint_new['date_diff']>0]

In [28]:
behind_month 

,FundId,Month,Fund Size,Fund Flow,esg_score,capm_Performance,performance_Three_Factor,performance_Four_Factor,Inception Date,Equity Style Box (Long),Age,size_log,esb_l,relative_flow,relative_flow_nxt,esg_performance,date_diff
46,FS00008SZD,2015-11-01,7.171247e+08,-3.341083e+06,49.22,-0.282636,-2.265871,-2.560219,2009-01-28,Large Growth,81,8.855595,2.0,-0.004659,0.035163,-0.139113,0.657098
47,FS00008SZD,2015-12-01,7.256647e+08,2.551669e+07,49.06,-1.395584,-1.529967,-0.128763,2009-01-28,Large Growth,82,8.860736,2.0,0.035163,-0.035286,-0.684673,1.642744
48,FS00008SZD,2016-01-01,6.873996e+08,-2.425526e+07,47.86,1.363006,0.930779,0.919008,2009-01-28,Large Growth,83,8.837209,2.0,-0.035286,-0.037014,0.652335,2.661246
49,FS00008SZD,2016-02-01,6.560494e+08,-2.428296e+07,47.78,0.881001,4.166935,3.107147,2009-01-28,Large Growth,84,8.816937,2.0,-0.037014,0.048628,0.420942,3.679747
50,FS00008SZD,2016-03-01,7.252638e+08,3.526795e+07,47.88,3.142115,4.904443,4.535098,2009-01-28,Large Growth,85,8.860496,2.0,0.048628,0.013182,1.504444,4.632539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301387,FSUSA09WNF,2019-04-01,8.137770e+06,-1.487095e+03,NaN,-3.067829,-3.546688,-4.150616,2008-11-11,Large Blend,124,6.910505,3.0,-0.000183,0.000101,NaN,41.627138
301388,FSUSA09WNF,2019-05-01,7.979041e+06,8.084123e+02,NaN,3.666930,4.017213,5.035920,2008-11-11,Large Blend,125,6.901951,3.0,0.000101,0.000238,NaN,42.612785
301389,FSUSA09WNF,2019-06-01,8.651099e+06,2.060187e+03,40.91,4.207674,3.772864,3.199166,2008-11-11,Large Blend,126,6.937071,3.0,0.000238,-0.000259,1.721360,43.631286
301390,FSUSA09WNF,2019-07-01,8.737163e+06,-2.267210e+03,NaN,1.134835,4.536452,4.007588,2008-11-11,Large Blend,127,6.941370,3.0,-0.000259,0.000258,NaN,44.616933


In [74]:
#catagorize the month - 2012/01 to 2019/08, totoally 92 months
month = pd.Categorical(joint_new.Month)

#use the 'Month' to divide the dataset, the index is Fund ID
df = joint_new.set_index(['FundId', 'Month'])
df['Month'] = month

In [75]:
df

Fund Size     Fund Flow  esg_score  \
FundId     Month                                               
FS00008SZD 2012-01-01  2.609599e+08  4.379443e+07        NaN   
           2012-02-01  2.988403e+08  2.572761e+07        NaN   
           2012-03-01  3.071028e+08  1.558167e+06        NaN   
           2012-04-01  3.440745e+08  3.753147e+07        NaN   
           2012-05-01  3.771187e+08  4.877692e+07        NaN   
...                             ...           ...        ...   
FSUSA09WNF 2019-04-01  8.137770e+06 -1.487095e+03        NaN   
           2019-05-01  7.979041e+06  8.084123e+02        NaN   
           2019-06-01  8.651099e+06  2.060187e+03      40.91   
           2019-07-01  8.737163e+06 -2.267210e+03        NaN   
           2019-08-01  8.255823e+06  2.131805e+03        NaN   

                       capm_Performance  performance_Three_Factor  \
FundId     Month                                                    
FS00008SZD 2012-01-01          1.507391                  3.652442   
           2012-02-01          1.544497                  1.220171   
           2012-03-01          4.025839                  0.078026   
           2012-04-01          2.647697                  0.989935   
           2012-05-01         -0.066138                 -0.807304   
...                                 ...                       ...   
FSUSA09WNF 2019-04-01         -3.067829                 -3.546688   
           2019-05-01          3.666930                  4.017213   
           2019-06-01          4.207674                  3.772864   
           2019-07-01          1.134835                  4.536452   
           2019-08-01         -5.479327                 -3.326145   

                       performance_Four_Factor Inception Date  \
FundId     Month                                                
FS00008SZD 2012-01-01                 1.841484     2009-01-28   
           2012-02-01                 1.155288     2009-01-28   
           2012-03-01                 0.963080     2009-01-28   
           2012-04-01                 1.171659     2009-01-28   
           2012-05-01                -0.354775     2009-01-28   
...                                        ...            ...   
FSUSA09WNF 2019-04-01                -4.150616     2008-11-11   
           2019-05-01                 5.035920     2008-11-11   
           2019-06-01                 3.199166     2008-11-11   
           2019-07-01                 4.007588     2008-11-11   
           2019-08-01                -2.716709     2008-11-11   

                      Equity Style Box (Long)  Age  size_log  esb_l  \
FundId     Month                                                      
FS00008SZD 2012-01-01            Large Growth   35  8.416574    2.0   
           2012-02-01            Large Growth   36  8.475439    2.0   
           2012-03-01            Large Growth   37  8.487284    2.0   
           2012-04-01            Large Growth   38  8.536653    2.0   
           2012-05-01            Large Growth   39  8.576478    2.0   
...                                       ...  ...       ...    ...   
FSUSA09WNF 2019-04-01             Large Blend  124  6.910505    3.0   
           2019-05-01             Large Blend  125  6.901951    3.0   
           2019-06-01             Large Blend  126  6.937071    3.0   
           2019-07-01             Large Blend  127  6.941370    3.0   
           2019-08-01             Large Blend  128  6.916760    3.0   

                       relative_flow  relative_flow_nxt  esg_performance  \
FundId     Month                                                           
FS00008SZD 2012-01-01       0.167821           0.086092              NaN   
           2012-02-01       0.086092           0.005074              NaN   
           2012-03-01       0.005074           0.109079              NaN   
           2012-04-01       0.109079           0.129341              NaN   
           2012-05-01       0.129341           0.027948              

In [76]:
#capm Performance summary
exog_vars = ['esg_score','size_log', 'Age', 'performance_Four_Factor','esg_performance']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.relative_flow, exog, entity_effects=True, time_effects=True)
mod_nxt = PanelOLS(df.relative_flow_nxt, exog, entity_effects=True, time_effects=True)

res = mod.fit()
res_nxt = mod_nxt.fit()

In [77]:
print(res_nxt)

                          PanelOLS Estimation Summary                           
Dep. Variable:      relative_flow_nxt   R-squared:                        0.0038
Estimator:                   PanelOLS   R-squared (Between):             -7.8687
No. Observations:              152797   R-squared (Within):              -0.0455
Date:                Thu, Apr 09 2020   R-squared (Overall):             -1.1170
Time:                        21:46:31   Log-likelihood                 2.191e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      114.88
Entities:                        3251   P-value                           0.0000
Avg Obs:                       47.000   Distribution:                F(5,149452)
Min Obs:                       1.0000                                           
Max Obs:                       90.000   F-statistic (robust):             114.88
                            

In [21]:
#catagorize the month - 2012/01 to 2019/08, totoally 92 months
month_front = pd.Categorical(front_month.Month)
month_behind = pd.Categorical(behind_month.Month)

#use the 'Month' to divide the dataset, the index is Fund ID
df_front = front_month.set_index(['FundId', 'Month'])
df_front['Month'] = month_front

df_behind = behind_month.set_index(['FundId', 'Month'])
df_behind['Month'] = month_behind

In [29]:
#robustness test
exog_vars_front = ['esg_score','size_log', 'Age', 'capm_Performance','esg_performance']
exog_front = sm.add_constant(df_front[exog_vars_front])

mod_front = PanelOLS(df_front.relative_flow_nxt, exog_front, entity_effects=True, time_effects=True)

res_front = mod_front.fit()


exog_vars_behind = ['esg_score','size_log', 'Age', 'capm_Performance','esg_performance']
exog_behind = sm.add_constant(df_behind[exog_vars_behind])

mod_behind = PanelOLS(df_behind.relative_flow_nxt, exog_behind, entity_effects=True, time_effects=True)

res_behind = mod_behind.fit()

In [30]:
print(res_front)

                          PanelOLS Estimation Summary                           
Dep. Variable:      relative_flow_nxt   R-squared:                        0.0073
Estimator:                   PanelOLS   R-squared (Between):             -16.218
No. Observations:               57280   R-squared (Within):              -0.0640
Date:                Fri, Apr 10 2020   R-squared (Overall):             -3.8844
Time:                        11:21:42   Log-likelihood                    4618.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      80.876
Entities:                        2142   P-value                           0.0000
Avg Obs:                       26.741   Distribution:                 F(5,55089)
Min Obs:                       1.0000                                           
Max Obs:                       45.000   F-statistic (robust):             80.876
                            

In [31]:
print(res_behind)

                          PanelOLS Estimation Summary                           
Dep. Variable:      relative_flow_nxt   R-squared:                        0.0049
Estimator:                   PanelOLS   R-squared (Between):             -0.5254
No. Observations:               95517   R-squared (Within):               0.0045
Date:                Fri, Apr 10 2020   R-squared (Overall):             -0.0824
Time:                        11:21:42   Log-likelihood                 2.008e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      89.996
Entities:                        3210   P-value                           0.0000
Avg Obs:                       29.756   Distribution:                 F(5,92258)
Min Obs:                       1.0000                                           
Max Obs:                       45.000   F-statistic (robust):             89.996
                            